# Section 3 : LightFM RecSys Model

 - This first attempt will involve treating the quantities as rating on its own without including theitem features

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
from pandas.api.types import CategoricalDtype
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
#pd.options.plotting.backend = 'plotly'

###################################################
import scipy.sparse as sparse
import random
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split

########################################################
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from itertools import islice

#######################################################


/home/admin1/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [269]:
recsys_df = pd.read_csv('Olaleke_Tasks/RecSys_df.csv')
recsys_df.head()

,CardHolder,Material,SalesCount,ItemPrice,net_weight,SubCategorycode,ItemDescription,Class,rpa_wgh2,rpa_wgh2_SubClass,rpa_wgh3,rpa_wgh3_SubClass,vendor
0,10008254,109900,3.0,20.97,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0
1,10011935,109900,1.0,6.49,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0
2,10036221,109900,3.0,19.47,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0
3,10036221,109900,2.0,12.98,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0
4,10036221,109900,4.0,25.96,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0


In [3]:
No_lentabag_df = pd.read_csv('Olaleke_Tasks/No_lentabag.csv')
No_lentabag_df.head()

,CardHolder,Material,SalesCount,ItemPrice,net_weight,SubCategorycode,ItemDescription,Class,rpa_wgh2,rpa_wgh2_SubClass,rpa_wgh3,rpa_wgh3_SubClass,vendor
0,10008254,439630,1.0,169.99,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
1,14919617,439630,1.0,109.89,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
2,15041309,439630,1.0,169.99,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
3,15041309,439630,1.0,116.99,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
4,15041309,439630,1.0,152.14,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0


In [4]:
#                                   ##groupby CustomerID && Item, then summing quantity values                   
# used_dff = recsys_df[['CardHolder','Material','SalesCount']]
# used_df = used_dff.groupby(['CardHolder','Material']).sum().reset_index()  

#                                   #replacing a sum of 0 with a sum of 1                               
# used_df.SalesCount.loc[used_df.SalesCount==0] = 1
#                                  #get only customers with positive num of quantities
# user_item_info = used_df.query('SalesCount > 0')  
# print(user_item_info.shape)
# user_item_info.head()

In [5]:
user_item_info = pd.read_csv('Olaleke_Tasks/user_item_nonbag.csv')
user_item_info.head()

,CardHolder,Material,SalesCount
0,10008254,4135,2.0
1,10008254,5383,2.0
2,10008254,5880,2.0
3,10008254,78790,2.0
4,10008254,81107,1.0


In [18]:
#user_item_info.to_csv('user_item_qqt.csv', index=False)

## Getting Train/Test Data and Interaction Matrix 

In [6]:
                             #spliting into train and test data...
                             #using 20% of data as 
train, test = train_test_split(user_item_info, test_size=0.2)    

In [7]:
print("Shape of the training data: ",train.shape)
print("Shape of the testing data: ", test.shape)

Shape of the training data:  (12164395, 3)
Shape of the testing data:  (3041099, 3)


In [8]:
def get_user_list(df,column_name):
    return np.sort(df[column_name].unique())

def get_item_list(df,column_name):
    return df[column_name].unique()

In [9]:
user_list = get_user_list(user_item_info,'CardHolder')
user_list

array([10008254, 10011587, 10011935, ..., 83332008, 83332870, 83333056])

In [10]:
item_list = get_item_list(user_item_info,'Material')
item_list

array([  4135,   5383,   5880, ..., 291484, 493272, 531239])

In [11]:
print("Number of unique customers: ",len(user_list))
print("Number of unique items: ", len(item_list))

Number of unique customers:  86497
Number of unique items:  58320


In [12]:
def id_mapping(user_list,item_list):
    user_to_index = {}                #convert user_id to index
    index_to_user = {}                #convert index to user...
    for user_index,user_id in enumerate(user_list):
        user_to_index[user_id] = user_index
        index_to_user[user_index] = user_id

    item_to_index = {}
    index_to_item = {}
    for item_index,item_id in enumerate(item_list):
        item_to_index[item_id] = item_index
        index_to_item[item_index] = item_id
  
    return user_to_index,index_to_user,\
         item_to_index,index_to_item

In [13]:
 user_to_index,index_to_user,\
         item_to_index,index_to_item = id_mapping(user_list,item_list)

In [14]:
                                    #function to view to dict elements..
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

n_items = take(10,item_to_index.items())   #view the 1st n items if a dictionary..
n_items

[(4135, 0),
 (5383, 1),
 (5880, 2),
 (78790, 3),
 (81107, 4),
 (143015, 5),
 (152314, 6),
 (177471, 7),
 (235949, 8),
 (252053, 9)]

In [15]:
def get_interaction_matrix(df,df_column_as_row,df_column_as_col,
              df_column_as_values,row_indexing_map,col_indexing_map):
    row = df[df_column_as_row].apply(lambda x: row_indexing_map[x]).values
    col = df[df_column_as_col].apply(lambda x: col_indexing_map[x]).values
    values = df[df_column_as_values].values 

    matrix_shape = (len(row_indexing_map),len(col_indexing_map))
    interaction_matrix = coo_matrix((values,(row,col)),shape = matrix_shape)
    return interaction_matrix 

In [16]:
train_interaction_matrix = get_interaction_matrix(train,'CardHolder','Material','SalesCount',
                                                  user_to_index,item_to_index)

test_interaction_matrix = get_interaction_matrix(test,'CardHolder','Material','SalesCount',
                                                  user_to_index,item_to_index)

## Model Training with LightFM

In [17]:
                    #modeling..
lightFM_model = LightFM(
    no_components = 150,
    learning_rate = 0.05,
    loss = 'warp',
    random_state = 42)

lightFM_model.fit(
    train_interaction_matrix,
    item_features = None,
    user_features = None,
    sample_weight = None,
    epochs=20,
    num_threads = 4,
    verbose = True
)                    

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [18]:
aucScore = auc_score(model = lightFM_model,
                      test_interactions = test_interaction_matrix,
                      num_threads = 4,
                      check_intersections = False)

In [19]:
aucScore.mean()

0.89611024

In [20]:
                  ##recreating the interaction matrix with the whole data
interaction_matrix = get_interaction_matrix(user_item_info,'CardHolder','Material','SalesCount',
                                                  user_to_index,item_to_index)                  

interaction_matrix

<86497x58320 sparse matrix of type '<class 'numpy.float64'>'
	with 15205494 stored elements in COOrdinate format>

In [21]:
                    #modeling..
lightFM_model = LightFM(
    no_components = 150,
    learning_rate = 0.05,
    loss = 'warp',
    random_state = 42)
                    #retraining the model with the whole dataset
lightFM_model.fit(
    interaction_matrix,
    item_features = None,
    user_features = None,
    sample_weight = None,
    epochs=20,
    num_threads = 4,
    verbose = True
)                    

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


## LightFM Model with Item features 

- This will combine both "user-item" interactions and the "item - feature" interactions


- its an hybrid method

In [22]:
features_dff = No_lentabag_df
features_dff.head(3)

,CardHolder,Material,SalesCount,ItemPrice,net_weight,SubCategorycode,ItemDescription,Class,rpa_wgh2,rpa_wgh2_SubClass,rpa_wgh3,rpa_wgh3_SubClass,vendor
0,10008254,439630,1.0,169.99,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
1,14919617,439630,1.0,109.89,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
2,15041309,439630,1.0,169.99,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0


In [23]:
features_dff.columns

Index(['CardHolder', 'Material', 'SalesCount', 'ItemPrice', 'net_weight',
       'SubCategorycode', 'ItemDescription', 'Class', 'rpa_wgh2',
       'rpa_wgh2_SubClass', 'rpa_wgh3', 'rpa_wgh3_SubClass', 'vendor'],
      dtype='object')

In [272]:
features_dff[['vendor']].nunique()

vendor    2385
dtype: int64

In [275]:
features_dff['rpa_wgh2_SubClass']= features_dff['rpa_wgh2_SubClass'].dropna()
features_dff['rpa_wgh2_SubClass'].nunique()

62

In [276]:
features_dff['rpa_wgh3_SubClass']= features_dff['rpa_wgh3_SubClass'].dropna()
features_dff['rpa_wgh3_SubClass'].nunique()

385

In [277]:
features_dff['Class'].nunique()

3

In [24]:
            #getting feature list
def get_feature_list(df,column_name1,column_name2,column_name3):
    
    feature1 = df[column_name1]
    feature2 = df[column_name2]
    feature3 = df[column_name3]
    
    feature_list = pd.concat([feature1,feature2,feature3], ignore_index = True).unique()
    return feature_list

def feature_id_mapping(feature_list):
    feature_to_index = {}                #convert user_id to index
    index_to_feature = {}                #convert index to user...
    for feature_index, feature_id in enumerate(feature_list):
        feature_to_index[feature_id] = feature_index
        index_to_feature[feature_index] = feature_id

    return feature_to_index,index_to_feature

In [25]:
feature_list = get_feature_list(features_dff,'Class','rpa_wgh2_SubClass','rpa_wgh3_SubClass')
feature_list = [x for x in feature_list if str(x) != 'nan']
np.array(feature_list)

array(['FOOD', 'NON FOOD', 'NONSALES', 'Pet goods',
       '"Household cleaning products "',
       'Dairy products and delicatessen', 'Services',
       'Cosmetics, perfume, hygiene, pharmaceuticals',
       'Cleaning and storage goods', 'Garden goods',
       "Women's and men's underwear", 'Condiments, spices, dressings',
       'Butter and fat products', 'Nonalcoholic beverages', 'Canned food',
       'Fresh poultry', 'Fresh and processed fish products',
       'Mayonnaise, Snacks, Ready meals Chilled', 'Cookware', 'Eggs',
       'Wine', 'Non chocolate Confectionery', 'Toys', 'Baby hygiena',
       'DIY', 'Electrical goods', 'Own production bakery', 'Cheese',
       'Own production culinary products', 'Coffee, cocoa, tea',
       'Frozen semi-finished and other products', 'Tobacco',
       'Commercial production bakery', 'Fresh meat', 'Fruits',
       'Fish and seafood delicatessen', 'Grocery', 'Vegetables',
       'Tableware, household goods', 'Functional food', 'Snacks',
       'C

In [26]:
feature_to_index,index_to_feature = feature_id_mapping(feature_list)
take(10,feature_to_index.items())

[('FOOD', 0),
 ('NON FOOD', 1),
 ('NONSALES', 2),
 ('Pet goods', 3),
 ('"Household cleaning products "', 4),
 ('Dairy products and delicatessen', 5),
 ('Services', 6),
 ('Cosmetics, perfume, hygiene, pharmaceuticals', 7),
 ('Cleaning and storage goods', 8),
 ('Garden goods', 9)]

In [27]:
                               ##get_item_feature_interaction
def get_items_features(df,f1_weight = 1,f2_weight = 1,f3_weight=1):

    items_class_df = df[["Material","Class"]].rename(columns = {"Class" : "feature"})
    items_class_df["feature_count"] = f1_weight 

    items_subclass2_df = df[["Material","rpa_wgh2_SubClass"]].rename(columns = {"rpa_wgh2_SubClass":"feature"})
    items_subclass2_df["feature_count"] = f2_weight 

    items_subclass3_df = df[["Material","rpa_wgh3_SubClass"]].rename(columns = {"rpa_wgh3_SubClass":"feature"})
    items_subclass3_df["feature_count"] = f3_weight 

    
    item_features_df = pd.concat([items_class_df, items_subclass2_df,items_subclass3_df], ignore_index=True)

    del items_class_df
    del items_subclass2_df
    del items_subclass3_df

    item_features_df = item_features_df.groupby(["Material", "feature"], as_index = False)["feature_count"].sum()  
    return item_features_df

In [28]:
item_features_df = get_items_features(features_dff)

In [29]:
item_features_df.head()

,Material,feature,feature_count
0,21,Advertisement,1561
1,21,Canned fish,1561
2,21,Canned food,1561
3,21,FOOD,3122
4,21,Services,1561


In [30]:
                                                       #generating item_to_feature interaction
item_features_interaction = get_interaction_matrix(item_features_df,"Material","feature",
                                                        "feature_count",item_to_index,feature_to_index)
item_features_interaction

<58320x449 sparse matrix of type '<class 'numpy.int64'>'
	with 174815 stored elements in COOrdinate format>

In [31]:
                    #modeling..
lightFM_model_features = LightFM(
    no_components = 150,
    learning_rate = 0.05,
    loss = 'warp',
    random_state = 42)

lightFM_model_features.fit(
    train_interaction_matrix,
    item_features = item_features_interaction,  
    user_features = None,
    sample_weight = None,
    epochs=20,
    num_threads = 4,
    verbose = True
)                    

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [32]:
auc_with_features = auc_score(model = lightFM_model_features, 
                        test_interactions = test_interaction_matrix,
                        train_interactions = train_interaction_matrix, 
                        item_features = item_features_interaction,
                        num_threads = 4, check_intersections=False)
auc_with_features.mean()

0.50429296

In [314]:
#                                               #function to generate unique interger id from the id in the dataframe
#                                               #this is important because LightFM only work with numeric id
# def generate_id(df,id_colum_name):            #id_colum_name : colum name of the new_id we want to generate new ones for
#     new_df = df.assign(int_id_col_name=np.arange(len(df))).reset_index(drop=True)    #assigning integer_id 
#     new_df = new_df.rename(columns={'int_id_col_name': id_colum_name})               #changing colum name 
#     return new_df 


#                                                        #function to create features to feed into lightFM
# def create_features(df,features_name, id_colum_name):  #feature_name:List of feature columns name avaiable in dataframe
#     features = df[features_name].apply(                  #id_col_name: String : Column name which contains id of the question or
#           lambda x: ','.join(x.map(str)),axis=1)           #...answer that the features will map to.                                    
#     features = features.str.split(',')                                  
#     features = list(zip(df[id_colum_name],features ))   
#     return features                                      #returns a pd series with features that can be fed-into the lightFM 
#                                                        #The format of each value will be (user_id, ['feature_1', 'feature_2', 'feature_3'])
#                                                        #Ex. -> (1, ['military', 'army', '5'])


# # new_ = generate_id(answer_scores, "id_num")
# # new_.head()

# users_df = recsys_df[['CardHolder','Material','vendor']]
# items_df = recsys_df[['Material','ItemDescription','ItemPrice',
#                       'net_weight','Class','rpa_wgh2_SubClass',
#                       'rpa_wgh3_SubClass']]

# users_df = generate_id(users_df,"user_id_num")
# items_df = generate_id(items_df,"item_id_num")
# items_df.head()

## Example: Making Recommendations for random users.

- The LightFm Model without the item features(pure collaborative model) performed better and will be used for making prediction

In [37]:
item_lookup_nonbag = pd.read_csv('Olaleke_Tasks/item_lookup_nonbag.csv')
item_lookup = item_lookup_nonbag
item_lookup.head()

,Material,ItemDescription
0,439630,Dry dog food up to 800 gr
1,338329,Dry dog food up to 800 gr
2,338283,Dry dog food up to 800 gr
3,338287,Dry dog food up to 800 gr
4,513215,Dry dog food up to 800 gr


In [38]:
                #MAKING RECOMMENDATION..
class recommendation_sampling:
    def __init__(self,model,items = item_list,
               user_item_iteraction_matrix = interaction_matrix,
               user_index_map = user_to_index):
        
        self.user_item_iteraction_matrix = user_item_iteraction_matrix
        self.model = model
        self.items = items
        self.user_index_map = user_index_map

    def recommend_items(self,user, num=10):
        userID = self.user_index_map.get(user,None)
        if userID == None:
              return None
        users = [userID]
        items_already_bought = self.items[self.user_item_iteraction_matrix.tocsr()[userID].indices] 
        scores = self.model.predict(user_ids = users,item_ids = np.arange(interaction_matrix.shape[1]))
    
        recommended_items = self.items[np.argsort(-scores)]
        recommend_list = []
        for i in recommended_items[:num]:
            recommend_list.append([i,item_lookup.ItemDescription.loc[item_lookup.Material == i].iloc[0]]) 
        
        codes = [item[0] for item in recommend_list]
        descriptions = [item[1] for item in recommend_list]
        final_frame = pd.DataFrame({'Material': codes, 'ItemDescription': descriptions})   #Create a dataframe 
        recommended = final_frame[['Material', 'ItemDescription']]                         #Switch order of columns around

        already_bought_list = []
        for i in items_already_bought[:num]:
            already_bought_list.append([i,item_lookup.ItemDescription.loc[item_lookup.Material == i].iloc[0]]) 
        
        codes_prev_items = [item[0] for item in already_bought_list]
        previous_items= [item[1] for item in already_bought_list]
        previous_item_df = pd.DataFrame({'Material': codes_prev_items, 'ItemDescription': previous_items})   #Create a dataframe 
        already_bought_df = previous_item_df[['Material', 'ItemDescription']]                         #Switch order of columns around
        

       
        compare_output = pd.concat([already_bought_df['ItemDescription'],
                                recommended['ItemDescription']], ignore_index=True, axis=1)
        compare_output = compare_output.rename(columns={0:"Already_Purchased_Items",
                           1:"Recommended_Items(lightFM)"})
        print("Recommendation for user_id: {}".format(user))
        return already_bought_df,recommended,compare_output
    

In [39]:
recom = recommendation_sampling(model = lightFM_model)

In [40]:
#user_to_index
take(10,user_to_index.items() )

[(10008254, 0),
 (10011587, 1),
 (10011935, 2),
 (10036221, 3),
 (10049978, 4),
 (10050032, 5),
 (10050960, 6),
 (10054537, 7),
 (10069565, 8),
 (10071527, 9)]

In [41]:
recom.recommend_items(10069565)[2]

Recommendation for user_id: 10069565


,Already_Purchased_Items,Recommended_Items(lightFM)
0,"""Special liquid washing detergent """,Bananas
1,Parsley,Lemons
2,Bananas,Bulb onion
3,Spicy greens,Spoonable yoghurts cold storage
4,Toothpaste,Bulk tangerines
5,Toothpaste,"""Bulk apples """
6,Other kitchenware,Bulk oranges
7,Bulk oranges,Carrot
8,"""Cabbage ""","""Sweet pepper """
9,Bulk traditional tomatoes,"Baguette, own production"


In [61]:
#user_list[-200:]

In [43]:
recom.recommend_items(83326911)[2]

Recommendation for user_id: 83326911


,Already_Purchased_Items,Recommended_Items(lightFM)
0,"""Smoked white fish slices in branded package """,Bananas
1,"""Cabbage """,Three-layer toilet paper
2,"""Liquid washing detergent """,Dill
3,Other grains,Bulb onion
4,Dishwasher tablets,"""Sweet pepper """
5,"""Carbonated wine drinks """,Toothpaste
6,Baby fabric conditioner,Baby fabric conditioner
7,"WC blocks, gel",Lemons
8,Toilet cleansers,"""Liquid washing detergent """
9,Beans,Household Bags


In [44]:
recom.recommend_items(83332008)[2]

Recommendation for user_id: 83332008


,Already_Purchased_Items,Recommended_Items(lightFM)
0,"""Vegetable salads with vegetable oil, own prod...",Female deodorant spray
1,Spring onion,Spoonable yoghurts cold storage
2,Cow pasteurized milk <1 l,Spoonable yoghurts cold storage
3,Cereals for children,Spoonable yoghurts cold storage
4,"""Table and drinking still water """,Spoonable yoghurts cold storage
5,Fresh vegetable snacks and cuts,Spoonable yoghurts cold storage
6,Female deodorant spray,Spoonable yoghurts cold storage
7,"""Flavoured cookies """,Spoonable yoghurts cold storage
8,Functional drinks,Spoonable yoghurts cold storage
9,Licensed beer in bottle,"Curd rolls, snacks cold storage"


In [45]:
recom.recommend_items(83332870)[2]

Recommendation for user_id: 83332870


,Already_Purchased_Items,Recommended_Items(lightFM)
0,"""Poultry salads with mayonnaise, own productio...",Other fast food products own production
1,Other fast food products own production,"Meat salads with mayonnaise, own production"
2,"Energy drinks < 0,6 l","""Hot poultry meals, own production """
3,NaN,"""Fish and seafood salads with mayonnaise, own ..."
4,NaN,"""Poultry salads with mayonnaise, own productio..."
5,NaN,"Side meals, vegetable and mushroom meals, own ..."
6,NaN,"""Fish and seafood salads with mayonnaise, own ..."
7,NaN,"Side meals, vegetable and mushroom meals, own ..."
8,NaN,"""Fish and seafood salads with mayonnaise, own ..."
9,NaN,"""Fish and seafood salads with mayonnaise, own ..."


In [46]:
recom.recommend_items(10054537)[2]

Recommendation for user_id: 10054537


,Already_Purchased_Items,Recommended_Items(lightFM)
0,Instant mashed potato,Fish fillet branded packaging
1,Fish fillet branded packaging,"Baguette, own production"
2,Cheese piece with colored mold,Lemons
3,NaN,Bulb onion
4,NaN,"""Fish cream and butter """
5,NaN,Bananas
6,NaN,Bulk cucumbers
7,NaN,Cooked smoked delicatessen
8,NaN,Carrot
9,NaN,"""Fish cream and butter """


In [52]:
recom.recommend_items(83285172)[2]

Recommendation for user_id: 83285172


,Already_Purchased_Items,Recommended_Items(lightFM)
0,Parsley,"Body creams, lotions"
1,Cleaning cloth,Toothpaste
2,Panty liners,"""Italian pasta recipes """
3,Three-layer toilet paper,Cleaning cloth
4,"""Italian pasta recipes """,Cleaning cloth
5,"""Face wash """,Cleaning cloth
6,Mouthwash,"WC blocks, gel"
7,"Body creams, lotions",Bar soap
8,NaN,Cleaning cloth
9,NaN,Men's shorts


In [53]:
recom.recommend_items(83284889)[2]

Recommendation for user_id: 83284889


,Already_Purchased_Items,Recommended_Items(lightFM)
0,"Washing detergent - capsules, tablets","Wheat bread, own production"
1,Bananas,Jar covers
2,Spicy greens,"Wheat bread, own production"
3,Toothpaste,Bulb onion
4,"""Sweet pepper """,National bread own production
5,Cotton wool,"Bread traditional wheat whole (bottom, molded)"
6,Lemons,Bulk tangerines
7,Washer fluid reservoir winter,Grass and vegetable seeds
8,Bulb onion,Canned peas
9,Coffee beans,"Baguette, own production"


In [54]:
recom.recommend_items(83285308)[2]

Recommendation for user_id: 83285308


,Already_Purchased_Items,Recommended_Items(lightFM)
0,Bananas,"Energy drinks < 0,6 l"
1,"Chicken eggs, more than 15 eggs",Cola-based lemonade
2,Bulk traditional tomatoes,Bananas
3,"""Sweet pepper """,A loaf of wheat in sliced.
4,Other root vegetables,Bulb onion
5,Carrot,A loaf of wheat in sliced.
6,"Grill, own production",A loaf of wheat in sliced.
7,Bulk potato,Long-term storage rolled cakes
8,"""Frozen one vegetable type mix, packaged ""","""Russian vodka """
9,Bulk potato,Cow sterilized milk (UHT) <1 l


In [55]:
recom.recommend_items(83286223)[2]

Recommendation for user_id: 83286223


,Already_Purchased_Items,Recommended_Items(lightFM)
0,"""Low-fat sterilized cream (below 15%) """,Bananas
1,Cow sterilized milk (UHT) <1 l,Bulb onion
2,Oatmeal cookies,Lemons
3,Bananas,Carrot
4,Condensed milk,Bulk tangerines
5,"""Sweet pepper """,Bulk traditional tomatoes
6,Lemons,"Wheat bread, own production"
7,"""Buckwheat """,Bulk potato
8,Bulb onion,Sand sugar white
9,Food use bags,Sand sugar white


In [56]:
recom.recommend_items(83286759)[2]

Recommendation for user_id: 83286759


,Already_Purchased_Items,Recommended_Items(lightFM)
0,Bulk oranges,Black olives
1,"""Vegetable salads with mayonnaise, own product...",Bulk tangerines
2,Bulk traditional tomatoes,Frankfurters in branded package
3,"""Sweet pepper ""","""Natural salmon caviar """
4,Lemons,"""Piece of salted salmon in branded package """
5,Fresh cut chicken in branded package,Bulk cucumbers
6,"""Bulk apples """,Bulk traditional tomatoes
7,"""Edam, Gouda, Tilsiter cheese sold by weight """,Bananas
8,Bulk tangerines,Black olives
9,Cards,Lemons


In [57]:
recom.recommend_items(83286571)[2]

Recommendation for user_id: 83286571


,Already_Purchased_Items,Recommended_Items(lightFM)
0,"""Meat by-products in original package """,Sand sugar white
1,Bulk traditional tomatoes,Mayonnaise classic packaging over 501 ml
2,"""Sweet pepper ""","""Edam, Gouda, Tilsiter cheese sold by weight """
3,Lemons,Individual portion food (pouch)
4,Dill,"Chicken eggs, more than 15 eggs"
5,Bulb onion,Refined sunflower oil
6,"""Edam, Gouda, Tilsiter cheese sold by weight ""","""Soft wheat pasta, Russia """
7,Olive oil Pomace,Bulb onion
8,Cola-based lemonade,Sand sugar white
9,Bulk tangerines,A loaf of wheat in sliced.


In [58]:
recom.recommend_items(83333056)[2]

Recommendation for user_id: 83333056


,Already_Purchased_Items,Recommended_Items(lightFM)
0,Boxed chocolate,Terry towel 50х100
1,Boxed chocolate,Boxed chocolate
2,Boxed chocolate,Terry towel 50х100
3,Boxed chocolate,Boxed chocolate
4,Boxed chocolate,Terry towel 70х150
5,Boxed chocolate,Terry towel 70х150
6,Batteries,Other terry products
7,Terry towel 70х150,Boxed chocolate
8,Terry towel 70х150,Paper bags
9,Other terry products,"""Frying pan lids and screens """


In [59]:
recom.recommend_items(83326249)[2]

Recommendation for user_id: 83326249


,Already_Purchased_Items,Recommended_Items(lightFM)
0,Drinking yoghurt for children older than 3 years,Table games
1,"""Table and drinking still water """,Modelling accessories
2,Table games,"""Table and drinking still water """
3,Table games,Drinking yoghurt for children older than 3 years
4,NaN,"""Plastic construction toys """
5,NaN,Milkshakes for children older than 3 years
6,NaN,Oral care for babies older than 3 years
7,NaN,Baby water
8,NaN,Table games
9,NaN,Chocolate eggs


In [60]:
recom.recommend_items(10008254)[2]

Recommendation for user_id: 10008254


,Already_Purchased_Items,Recommended_Items(lightFM)
0,Canned herring,Individual portion food (pouch)
1,"""Canned scombrel """,Individual portion food (pouch)
2,Mayonnaise olive packaging up to 250 ml,Individual portion food (pouch)
3,"""Low-fat sterilized cream (below 15%) """,Bulb onion
4,"""Milk shakes warm storage """,Individual portion food (pouch)
5,"Chicken eggs, more than 15 eggs",Individual portion food (pouch)
6,"""Special liquid washing detergent """,Individual portion food (pouch)
7,"Bottled juice 0,6-1,4 l","Boneless pork large piece own production, service"
8,Butter packing 125 gr - 300 gr,Lemons
9,"""Vegetable paste """,Treats for cats
